In [15]:
using Pkg
Pkg.activate(@__DIR__);
pkg"instantiate"
pkg"precompile"

using RigidBodyDynamics
using MeshCatMechanisms

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/FlyHopper/scripts/Julia/Project.toml`


In [20]:
curdir = pwd()
const URDFPATH = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
bot = parse_urdf(URDFPATH, floating=true)
remove_fixed_tree_joints!(bot);

In [10]:
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
q = configuration(state)
v = velocity(state)

8-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
center_of_mass(state)

Point3D in "world": [-0.002547131739826878, -3.3470181064453804e-8, -0.0029411827079611035]

In [12]:
q0, q1 = joints(bot)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    # velocity_range: Return the range of indices into the joint velocity vector v corresponding to joint "joint"
    torques[velocity_range(state, q0)] .= -1 .* velocity(state, q0)
    torques[velocity_range(state, q1)] .= -1 .* velocity(state, q1)
end;

In [13]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

In [17]:
zero_velocity!(state)
# set_configuration!(state, q0, 0.7)
# set_configuration!(state, q1, -0.8);
mvis = MechanismVisualizer(bot, URDFVisuals(URDFPATH));

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8701
└ @ MeshCat /home/ben/.julia/packages/MeshCat/X2AUA/src/visualizer.jl:73


In [19]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);